In [2]:
import boto3
import os
import matplotlib.pyplot as plt
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch , PyTorchModel , PyTorchPredictor
from sagemaker.inputs import TrainingInput
from sagemaker.debugger import (Rule , 
                                rule_configs , 
                                ProfilerConfig ,
                                ProfilerRule ,
                                DebuggerHookConfig ,
                                FrameworkProfile ,
                               )
from sagemaker.predictor import Predictor

In [3]:
rule = get_execution_role()
train_s3_path = "s3://sagemaker-us-east-1-644236161621/train/"
valid_s3_path = "s3://sagemaker-us-east-1-644236161621/valid/"
train_input = TrainingInput(s3_data=train_s3_path)
valid_input = TrainingInput(s3_data=valid_s3_path)

In [4]:
rules = [Rule.sagemaker(rule_configs.vanishing_gradient()),
         Rule.sagemaker(rule_configs.overtraining()),
         Rule.sagemaker(rule_configs.overfit()),
         Rule.sagemaker(rule_configs.class_imbalance()),
         ProfilerRule.sagemaker(rule_configs.CPUBottleneck()),
         ProfilerRule.sagemaker(rule_configs.ProfilerReport())]
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "3", "eval.save_interval": "5"}
)

In [19]:
estimator  = PyTorch(entry_point="train.py",
                    role=get_execution_role(),
                    instance_count=1,
                    instance_type="ml.g4dn.xlarge",
                    framework_version="1.12",
                    py_version="py38" ,
                    ules=rules,
                    debugger_hook_config=hook_config ,
                    profiler_config=profiler_config)
estimator.fit({"train":train_input , "valid":valid_input})

2022-12-16 12:19:32 Starting - Starting the training job...
2022-12-16 12:19:55 Starting - Preparing the instances for trainingProfilerReport-1671193172: InProgress
.....................
2022-12-16 12:23:16 Downloading - Downloading input data...
2022-12-16 12:24:01 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-16 12:24:04,047 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-16 12:24:04,070 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2022-12-16 12:24:04,083 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-16 12:24:04,088 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-16 12:24:04,342 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed

In [20]:
estimator.model_data

's3://sagemaker-us-east-1-644236161621/pytorch-training-2022-12-16-12-19-31-997/output/model.tar.gz'

In [5]:
from sagemaker.serializers import JSONSerializer, IdentitySerializer , NumpySerializer
from sagemaker.deserializers import JSONDeserializer


In [34]:
x = torch.rand(size=(1,4))
x.argmax(dim=1).item()

2

In [6]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()

In [7]:
session = sagemaker.Session()
endpoint_name = "Weather-prediction-v6"
model_data = 's3://sagemaker-us-east-1-644236161621/pytorch-training-2022-12-16-12-19-31-997/output/model.tar.gz'#estimator.model_data
role = sagemaker.get_execution_role()

In [11]:
class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [26]:
model = PyTorchModel(model_data=model_data, role=role, entry_point='inference.py',py_version='py38',
                             framework_version='1.12',
                             predictor_cls=ImagePredictor)

In [27]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large' , endpoint_name=endpoint_name)

------!

In [32]:
#anthor one for numpy serelizer 
endpoint_name = "Weather-prediction-v9-numpy"
numpy_serelizer = NumpySerializer()
class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=numpy_serelizer,
            deserializer=json_deserializer,
        )
model2 = PyTorchModel(model_data=model_data, role=role, entry_point='inference2.py',py_version='py38',
                             framework_version='1.12',
                             predictor_cls=ImagePredictor)  
predictor2 = model2.deploy(initial_instance_count=1, instance_type='ml.m5.large' , endpoint_name=endpoint_name)

------!

In [21]:
type(predictor2)

__main__.ImagePredictor

In [109]:
test_image =r"train/cloudy1.jpg"
import io
import PIL
with open(test_image , "rb") as f:
    image = f.read()
    #print(isinstance(image.decode('utf-8') ,bytes))
    image = PIL.Image.open(io.BytesIO(image))
    image = torchvision.transforms.Compose([torchvision.transforms.Resize(200) ,
                                            torchvision.transforms.ToTensor()])(image)
    print(type(image))

<class 'torch.Tensor'>


In [ ]:
test_image =r"train/sunrise108.jpg"
with open(test_image , "rb") as f:
    response =predictor.predict(f.read() , initial_args={"ContentType": "image/jpeg"})
print(response)

{"type": "sunrise"}


In [16]:
test_image =r"train/sunrise108.jpg"
import json
import base64

In [17]:
with open(test_image , "rb") as f :
    data = f.read()
print(type(data))

<class 'bytes'>


In [18]:
endpoint_name = "Weather-prediction-v5"
import boto3
runtime=boto3.Session().client('sagemaker-runtime')
response=runtime.invoke_endpoint(EndpointName=endpoint_name,
                                    ContentType="image/jpeg",
                                    Body=data)
                                    

In [19]:
res = response['Body'].read().decode("utf-8")

In [20]:
out = json.loads(res)

In [21]:
json.loads(out)["type"]

'sunrise'

In [16]:
np.array(bytearray(data)).shape

(151200,)

In [33]:
test_image ="train/sunrise109.jpg"
data = plt.imread(test_image)
response =predictor2.predict(bytearray(data) , initial_args={"ContentType": "application/x-npy"})
print(response)

{"type": "shine"}
